## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

**1= fake news**,
**0= non-fake news**

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
## Drop NaN values
df=df.dropna()

In [5]:
## Get the independent features
X=df.drop('label', axis=1)

In [6]:
## Get the Dependent features
Y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
Y.shape

(18285,)

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.3.1'

In [18]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense

In [19]:
### Vocabulary size
voc_size=5000

### OneHot Representation

In [20]:
messages=X.copy()

In [21]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [22]:
messages.reset_index(inplace=True)

In [23]:
import nltk
import re # regula expression
from nltk.corpus import stopwords

In [25]:
## Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]

for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]'," ",messages['title'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [28]:
corpus[1:10]

['flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [30]:
## OneHot Encoding
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr[1:10]

[[4962, 1035, 4499, 2633, 1102, 382, 1900],
 [4763, 3727, 1136, 1102],
 [4147, 3924, 3153, 4743, 4499, 4700],
 [2333, 1102, 884, 604, 3267, 1328, 1102, 1918, 1614, 3361],
 [463,
  3473,
  2363,
  4869,
  3046,
  4360,
  3204,
  2012,
  2191,
  4217,
  802,
  1386,
  3533,
  1988,
  1900],
 [840, 4646, 1359, 1802, 1544, 4649, 3838, 333, 807, 1785, 3464],
 [1180, 204, 927, 1590, 4749, 4458, 4360, 243, 807, 1785, 3464],
 [314, 3347, 1977, 2734, 1977, 1050, 2284, 1792, 4360, 4476],
 [4067, 4162, 397, 1458, 3043, 1150, 2253, 4806]]

### Embedding Representation

In [31]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

print(embedded_docs)

[[   0    0    0 ...  318 4855 3963]
 [   0    0    0 ... 1102  382 1900]
 [   0    0    0 ... 3727 1136 1102]
 ...
 [   0    0    0 ...  807 1785 3464]
 [   0    0    0 ... 3371  421  165]
 [   0    0    0 ... 4541 1902 1805]]


In [33]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4544,
       2712, 4451, 3868, 3031, 3651,  173,  318, 4855, 3963], dtype=int32)

### Creating Model

In [36]:
embedding_vector_features=40

model=Sequential()

model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))

model.add(LSTM(100))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
len(embedded_docs),Y.shape

(18285, (18285,))

In [41]:
import numpy as np
X_final=np.array(embedded_docs)
Y_final=np.array(Y)

In [42]:
X_final.shape,Y_final.shape

((18285, 20), (18285,))

In [44]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_final,Y_final,
                                                     test_size=0.33,random_state=42)

## Model Training

In [45]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),
          epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 25ms/step - loss: 0.3352 - accuracy: 0.8432 - val_loss: 0.1986 - val_accuracy: 0.9152
Epoch 2/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1384 - accuracy: 0.9455 - val_loss: 0.2091 - val_accuracy: 0.9198
Epoch 3/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0938 - accuracy: 0.9661 - val_loss: 0.2300 - val_accuracy: 0.9223
Epoch 4/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0587 - accuracy: 0.9809 - val_loss: 0.2401 - val_accuracy: 0.9176
Epoch 5/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0351 - accuracy: 0.9891 - val_loss: 0.3217 - val_accuracy: 0.9142
Epoch 6/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0216 - accuracy: 0.9933 - val_loss: 0.3814 - val_accuracy: 0.9122
Epoch 7/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0133 - accuracy: 0.9956 - val_loss: 0.4322 - val_accuracy: 0.9095

### Model with added Dropout

In [46]:
from tensorflow.keras.layers import Dropout

In [52]:
## Creating model
embedding_vector_features=40

model_1=Sequential()

model_1.add(Embedding(voc_size,embedding_vector_features,
                    input_length=sent_length))

model_1.add(Dropout(0.3))

model_1.add(LSTM(100))

model_1.add(Dropout(0.3))

model_1.add(Dense(1,activation='sigmoid'))

model_1.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])

In [53]:
model_1.fit(X_train,Y_train,validation_data=(X_test,Y_test),
          epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 29ms/step - loss: 0.3519 - accuracy: 0.8376 - val_loss: 0.2022 - val_accuracy: 0.9140
Epoch 2/10
192/192 [==============================] - 5s 26ms/step - loss: 0.1491 - accuracy: 0.9416 - val_loss: 0.1911 - val_accuracy: 0.9191
Epoch 3/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1082 - accuracy: 0.9606 - val_loss: 0.2093 - val_accuracy: 0.9155
Epoch 4/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0840 - accuracy: 0.9699 - val_loss: 0.2257 - val_accuracy: 0.9175
Epoch 5/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0671 - accuracy: 0.9767 - val_loss: 0.2632 - val_accuracy: 0.9145
Epoch 6/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0499 - accuracy: 0.9838 - val_loss: 0.2755 - val_accuracy: 0.9153
Epoch 7/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0374 - accuracy: 0.9887 - val_loss: 0.2987 - val_accuracy: 0.9147

## Performance Metrics and Accuracy

### For Mdel --> without Dropout

In [54]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [55]:
from sklearn.metrics import confusion_matrix

In [57]:
confusion_matrix(Y_test,y_pred)

array([[2723,  696],
       [1952,  664]])

In [58]:
from sklearn.metrics import accuracy_score

In [60]:
accuracy_score(Y_test,y_pred)

0.5612261806130903

### For Model --> with Dropout

In [61]:
y_pred=model_1.predict_classes(X_test)

In [62]:
## Confusion matrix
confusion_matrix(Y_test,y_pred)

array([[3111,  308],
       [ 232, 2384]])

In [64]:
## Accuracy score
accuracy_score(Y_test,y_pred)

0.9105219552609777